# 03 — Decision Tree Model  
Hotel Booking Demand (Cancellation Prediction)

## Notebook purpose
This notebook develops a strong supervised baseline using **Decision Tree** and demonstrates:
- leakage-safe end-to-end modeling using `sklearn.Pipeline` (`preprocessing → model`)
- hyperparameter tuning using stratified cross-validation
- test-set evaluation using multiple metrics and diagnostic plots
- interpretability using coefficient analysis on transformed features

## Inputs
- Preferred: `data/processed/hotel_bookings_dedup.csv`
- Fallback: `data/raw/hotel_bookings.csv`

## Outputs (overwritten on each run)
- Model:
  - `artifacts/models/dt_pipeline.joblib`
- Metrics:
  - `artifacts/metrics/dt_cv_results.csv`
  - `artifacts/metrics/dt_best_params.json`
  - `artifacts/metrics/dt_test_metrics.json`
  - `artifacts/metrics/dt_threshold_metrics.csv`
  - `artifacts/metrics/dt_coefficients_top.csv`
- Plots:
  - `artifacts/plots/dt_confusion_matrix.png`
  - `artifacts/plots/dt_roc_curve.png`
  - `artifacts/plots/dt_pr_curve.png`
  - `artifacts/plots/dt_feature_importance.png`
  - `artifacts/plots/dt_tree_depth_vs_f1.png`
- Reports:
  - `artifacts/reports/dt_classification_report.txt`
  - `artifacts/reports/dt_notes.md`
  - `artifacts/reports/run_metadata.json`

All preprocessing steps are fitted on training folds only (no leakage).


In [1]:
# Repository bootstrap (fixes ModuleNotFoundError: 'src')
# The repository root is resolved quickly using Git when available.
# A bounded parent-directory scan is used as a fallback.

import os
import sys
import subprocess
from pathlib import Path

def _find_repo_root(max_levels: int = 25) -> Path:
    try:
        out = subprocess.check_output(
            ["git", "rev-parse", "--show-toplevel"],
            stderr=subprocess.DEVNULL,
            text=True,
        ).strip()
        p = Path(out)
        if (p / "src").is_dir():
            return p
    except Exception:
        pass

    cwd = Path.cwd()
    for p in [cwd] + list(cwd.parents)[:max_levels]:
        if (p / "src").is_dir():
            return p

    raise FileNotFoundError(
        "Folder 'src' was not found within the parent directories. "
        "Open the repository root folder in VS Code and rerun the notebook."
    )

root = _find_repo_root(max_levels=25)
os.chdir(root)
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print("Working directory:", Path.cwd())
print("Python path entry added:", root)


Working directory: D:\Documents\SLIIT_Campus\Y4S2\ML_SE40\ML_assignment\Machine-Learning-Assignment
Python path entry added: D:\Documents\SLIIT_Campus\Y4S2\ML_SE40\ML_assignment\Machine-Learning-Assignment


## Imports and configuration fallback

`src.config` may differ between group members during active development.
To prevent notebook failures, configuration values are loaded with safe fallbacks.


In [18]:
# Imports, config, artifacts
import platform
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.io_utils import ensure_artifact_dirs, save_json, save_text, save_dataframe, save_model, save_run_metadata
from src.data_loader import load_hotel_bookings, basic_train_ready_checks
from src.preprocessing import build_preprocessor, PreprocessOptions, get_feature_names
from src.metrics import compute_classification_metrics
from src.train_eval import (
    split_xy,
    train_test_split,
    get_estimator,
    build_model_pipeline,
    tune_with_gridsearch,
    evaluate_on_test,
    TrainOptions,
)
from src.plots import plot_confusion_matrix, plot_roc_curve, plot_pr_curve, plot_feature_importance
import src.config as cfg

warnings.filterwarnings("ignore", category=UserWarning)

PROJECT_NAME = getattr(cfg, "PROJECT_NAME", "ML-assignment")
RANDOM_STATE = int(getattr(cfg, "RANDOM_STATE", 42))
TARGET_COL = getattr(cfg, "TARGET_COL", "is_canceled")
TEST_SIZE = float(getattr(cfg, "TEST_SIZE", 0.20))
CV_SPLITS = int(getattr(cfg, "CV_SPLITS", 5))
LEAKAGE_COLS = list(getattr(cfg, "LEAKAGE_COLS", ["reservation_status", "reservation_status_date"]))
DEFAULT_DATA_PATH = str(getattr(cfg, "DEFAULT_DATA_PATH", "data/raw/hotel_bookings.csv"))
FORCE_CATEGORICAL_COLS = list(getattr(cfg, "FORCE_CATEGORICAL_COLS", ["agent", "company"]))
ONEHOT_MIN_FREQUENCY = getattr(cfg, "ONEHOT_MIN_FREQUENCY", 0.01)
PARAM_GRIDS = getattr(cfg, "PARAM_GRIDS", {}) or {}
FIG_DPI = getattr(cfg, "FIG_DPI", 160)

ART = ensure_artifact_dirs("artifacts")

meta_path = save_run_metadata(
    {
        "project": PROJECT_NAME,
        "random_state": RANDOM_STATE,
        "test_size": TEST_SIZE,
        "cv_splits": CV_SPLITS,
        "target_col": TARGET_COL,
        "notebook": "05_model_decision_tree.ipynb",
        "python_version": sys.version,
        "platform": platform.platform(),
    },
    base_dir="artifacts",
    repo_root=".",
)
print("Metadata file:", meta_path.resolve())
print("Artifacts base:", ART["base"].resolve())

display(pd.DataFrame([{
    "PROJECT_NAME": PROJECT_NAME,
    "RANDOM_STATE": RANDOM_STATE,
    "TEST_SIZE": TEST_SIZE,
    "CV_SPLITS": CV_SPLITS,
    "TARGET_COL": TARGET_COL,
    "ONEHOT_MIN_FREQUENCY": ONEHOT_MIN_FREQUENCY,
    "LEAKAGE_COLS": ", ".join(LEAKAGE_COLS),
    "FORCE_CATEGORICAL_COLS": ", ".join(FORCE_CATEGORICAL_COLS),
}]))


# Reduce warning noise during cross-validation
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r".*'penalty' was deprecated.*",
)


Metadata file: D:\Documents\SLIIT_Campus\Y4S2\ML_SE40\ML_assignment\Machine-Learning-Assignment\artifacts\reports\run_metadata.json
Artifacts base: D:\Documents\SLIIT_Campus\Y4S2\ML_SE40\ML_assignment\Machine-Learning-Assignment\artifacts


,PROJECT_NAME,RANDOM_STATE,TEST_SIZE,CV_SPLITS,TARGET_COL,ONEHOT_MIN_FREQUENCY,LEAKAGE_COLS,FORCE_CATEGORICAL_COLS
0,ML-assignment,42,0.2,5,is_canceled,0.01,"reservation_status, reservation_status_date","agent, company"


## Dataset loading

The processed dataset is preferred when available to standardize rows across the team.


In [19]:
preferred_processed = Path("data/processed/hotel_bookings_dedup.csv")
preferred_raw = Path(DEFAULT_DATA_PATH)

dataset_path = preferred_processed if preferred_processed.exists() else preferred_raw
if not dataset_path.exists():
    raise FileNotFoundError("Dataset not found. Expected at data/raw/hotel_bookings.csv")

print("Dataset path:", dataset_path.resolve())

df = load_hotel_bookings(dataset_path, drop_duplicates=False, verbose=True)
basic_train_ready_checks(df, target_col=TARGET_COL)

display(df.head())
display(pd.DataFrame({"rows": [df.shape[0]], "columns": [df.shape[1]]}))


Dataset path: D:\Documents\SLIIT_Campus\Y4S2\ML_SE40\ML_assignment\Machine-Learning-Assignment\data\processed\hotel_bookings_dedup.csv
[data_loader] Loaded shape: (87396, 32)
[data_loader] Columns: 32


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


,rows,columns
0,87396,32


## Type safety for identifier-like columns

Columns such as `agent` and `company` represent identifiers.
Casting them to string avoids implying a numeric ordering and supports one-hot encoding.


In [20]:
df_model = df.copy()
converted = []
for c in FORCE_CATEGORICAL_COLS:
    if c in df_model.columns:
        # Preserve missingness explicitly as a category label
        df_model[c] = pd.to_numeric(df_model[c], errors="coerce").astype("Int64").astype("string")
        df_model[c] = df_model[c].fillna("MISSING")
        converted.append(c)

display(pd.DataFrame({"converted_to_string": converted}))
if converted:
    display(df_model[converted].head())


,converted_to_string
0,agent
1,company


,agent,company
0,MISSING,MISSING
1,MISSING,MISSING
2,MISSING,MISSING
3,304,MISSING
4,240,MISSING


## Train/test split

A stratified split preserves cancellation rate in both partitions.
Split metadata and label distributions are displayed and saved.


In [21]:
X = df_model.drop(columns=[TARGET_COL])
y = df_model[TARGET_COL].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y,
)

split_info = {
    "n_rows_total": int(len(df_model)),
    "n_train": int(len(X_train)),
    "n_test": int(len(X_test)),
    "test_size": float(TEST_SIZE),
    "random_state": int(RANDOM_STATE),
    "stratified": True,
}
save_json(split_info, ART["data"] / "train_test_split_dt.json")

train_dist = y_train.value_counts().rename_axis("label").reset_index(name="count")
train_dist["rate"] = train_dist["count"] / train_dist["count"].sum()
test_dist = y_test.value_counts().rename_axis("label").reset_index(name="count")
test_dist["rate"] = test_dist["count"] / test_dist["count"].sum()

display(pd.DataFrame([split_info]))
display(train_dist)
display(test_dist)

print("Saved:", (ART["data"] / "train_test_split_dt.json").resolve())


,n_rows_total,n_train,n_test,test_size,random_state,stratified
0,87396,69916,17480,0.2,42,True


,label,count,rate
0,0,50696,0.725099
1,1,19220,0.274901


,label,count,rate
0,0,12675,0.725114
1,1,4805,0.274886


Saved: D:\Documents\SLIIT_Campus\Y4S2\ML_SE40\ML_assignment\Machine-Learning-Assignment\artifacts\data\train_test_split_dt.json


## Build preprocessors and model pipeline

In [23]:
preprocess_opts = PreprocessOptions(
    onehot_min_frequency=ONEHOT_MIN_FREQUENCY,
    output_sparse=True,
    scale_numeric=True,
)
preprocess = build_preprocessor(
    drop_cols=LEAKAGE_COLS,
    force_categorical_cols=FORCE_CATEGORICAL_COLS,
    options=preprocess_opts,
)

estimator = get_estimator("decision_tree", random_state=RANDOM_STATE)
pipeline = build_model_pipeline(preprocess, estimator)

pipeline

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('features', ...), ('transform', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,drop_cols,"['reservation_status', 'reservation_status_date']"
,force_categorical_cols,"['agent', 'company']"
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer an

## Define a reasonable hyperparameter grid (tweak as needed)

In [28]:
param_grid = PARAM_GRIDS.get(
    "decision_tree",
    {
        "model__max_depth": [None, 5, 10, 20],
        "model__min_samples_split": [2, 10, 30],
        "model__min_samples_leaf": [1, 5, 10],
        "model__class_weight": [None, "balanced"],
    },
)


# Hyperparameter Grid search CV
cv_options = TrainOptions(
    random_state=RANDOM_STATE,
    cv_splits=CV_SPLITS,
    scoring="f1",
    n_jobs=-1,
    verbose=2,
)
search = tune_with_gridsearch(
    pipeline=pipeline,
    param_grid=param_grid,
    X_train=X_train,
    y_train=y_train,
    options=cv_options,
)


# Save CV results and best params
save_dataframe(pd.DataFrame(search.cv_results_), ART["metrics"] / "dt_cv_results.csv", index=False)
save_json(search.best_params_, ART["metrics"] / "dt_best_params.json")
best_model = search.best_estimator_
print("Best params:", search.best_params_)



Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best params: {'model__class_weight': 'balanced', 'model__max_depth': 20, 'model__min_samples_leaf': 10, 'model__min_samples_split': 30}


In [29]:
import sklearn
from packaging.version import Version
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from tqdm.auto import tqdm

sk_ver = Version(sklearn.__version__)
print("scikit-learn version:", sklearn.__version__)

# Default DT grid (used if config has no decision_tree grid)
default_grid = {
    "model__max_depth": [None, 5, 10, 20],
    "model__min_samples_split": [2, 10, 30],
    "model__min_samples_leaf": [1, 5, 10],
    "model__class_weight": [None, "balanced"],
}

# Use config grid if defined; otherwise fallback to default
param_grid = PARAM_GRIDS.get("decision_tree", default_grid)
grid = list(ParameterGrid(param_grid))

display(pd.DataFrame(grid).head(15))
print("Parameter combinations:", len(grid))

cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)

def make_fresh_pipeline() -> Pipeline:
    """Create a fresh preprocessing→model pipeline for each fit (clone-safe)."""
    preprocess = build_preprocessor(
        drop_cols=LEAKAGE_COLS,
        force_categorical_cols=FORCE_CATEGORICAL_COLS,
        options=PreprocessOptions(
            onehot_min_frequency=ONEHOT_MIN_FREQUENCY,
            output_sparse=True,
            scale_numeric=True,
        ),
    )
    dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
    return Pipeline([("preprocess", preprocess), ("model", dt)])

results = []
best_so_far = -1.0
best_params = None

total_fits = len(grid) * CV_SPLITS
pbar = tqdm(total=total_fits, desc="DecisionTree CV fits", leave=True)

for params in grid:
    fold_scores = []
    for tr_idx, va_idx in cv.split(X_train, y_train):
        m = make_fresh_pipeline()
        m.set_params(**params)
        m.fit(X_train.iloc[tr_idx], y_train.iloc[tr_idx])
        pred = m.predict(X_train.iloc[va_idx])
        fold_scores.append(f1_score(y_train.iloc[va_idx], pred, zero_division=0))
        pbar.update(1)

    mean_f1 = float(np.mean(fold_scores))
    std_f1 = float(np.std(fold_scores))

    if mean_f1 > best_so_far:
        best_so_far = mean_f1
        best_params = dict(params)

    status = {
        "best_f1": f"{best_so_far:.4f}",
        "curr_f1": f"{mean_f1:.4f}",
        "max_depth": params.get("model__max_depth", ""),
        "min_split": params.get("model__min_samples_split", ""),
        "min_leaf": params.get("model__min_samples_leaf", ""),
        "cw": params.get("model__class_weight", ""),
    }
    pbar.set_postfix(status)

    results.append({
        **params,
        "cv_f1_mean": mean_f1,
        "cv_f1_std": std_f1,
        "cv_f1_scores": [float(x) for x in fold_scores],
    })

pbar.close()

# Results table, sorted
cv_results = pd.DataFrame(results).sort_values("cv_f1_mean", ascending=False).reset_index(drop=True)

# Persist results (scores column dropped for CSV brevity)
save_dataframe(cv_results.drop(columns=["cv_f1_scores"]), ART["metrics"] / "dt_cv_results.csv", index=False)
save_json(best_params, ART["metrics"] / "dt_best_params.json")

print("Saved:", (ART["metrics"] / "dt_cv_results.csv").resolve())
print("Best params:", best_params)

display(cv_results.head(15))

# Fit best model on full train
best_model = make_fresh_pipeline()
best_model.set_params(**best_params)
best_model.fit(X_train, y_train)

scikit-learn version: 1.8.0


,model__class_weight,model__max_depth,model__min_samples_leaf,model__min_samples_split
0,NaN,NaN,1,2
1,NaN,NaN,1,10
2,NaN,NaN,1,30
3,NaN,NaN,5,2
4,NaN,NaN,5,10
5,NaN,NaN,5,30
6,NaN,NaN,10,2
7,NaN,NaN,10,10
8,NaN,NaN,10,30
9,NaN,5.0,1,2


Parameter combinations: 72


DecisionTree CV fits:   3%|▎         | 12/360 [00:18<08:49,  1.52s/it, best_f1=0.6352, curr_f1=0.6346, max_depth=None, min_split=10, min_leaf=1, cw=None]

KeyboardInterrupt: 

In [ ]:
# 4) Test-set evaluation
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, "predict_proba") else None

test_metrics = compute_classification_metrics(y_test, y_pred, y_proba=y_proba)
save_json(test_metrics, ART["metrics"] / "dt_test_metrics.json")
print(test_metrics)

In [ ]:
# 5) Plots
plot_confusion_matrix(
    y_test, y_pred,
    out_path=ART["plots"] / "dt_confusion_matrix.png",
    dpi=FIG_DPI,
)
if y_proba is not None:
    plot_roc_curve(
        y_test, y_proba,
        out_path=ART["plots"] / "dt_roc_curve.png",
        dpi=FIG_DPI,
    )
    plot_pr_curve(
        y_test, y_proba,
        out_path=ART["plots"] / "dt_pr_curve.png",
        dpi=FIG_DPI,
    )


In [ ]:
# 6) Feature importance
feature_names = get_feature_names(best_model.named_steps["preprocess"], input_features=X_train.columns)
plot_feature_importance(
    model=best_model.named_steps["model"],
    feature_names=feature_names,
    top_n=30,
    out_path=ART["plots"] / "dt_feature_importance.png",
    dpi=FIG_DPI,
)

In [ ]:
# 7) Save fitted pipeline
save_model(best_model, ART["models"] / "dt_pipeline.joblib")

In [ ]:
# 8) (Optional) depth vs F1 diagnostic
depths = [2, 4, 6, 8, 10, 12, None]
rows = []
for d in depths:
    est = get_estimator("decision_tree", random_state=RANDOM_STATE)
    est.set_params(max_depth=d)
    pipe = build_model_pipeline(preprocess, est)
    pipe.fit(X_train, y_train)
    y_hat = pipe.predict(X_test)
    rows.append({"max_depth": d, "f1": f1_score(y_test, y_hat)})
pd.DataFrame(rows).to_csv(ART["metrics"] / "dt_tree_depth_vs_f1.csv", index=False)